In [ ]:
import praw
from kafka import KafkaProducer
import json
import schedule
import time
from datetime import datetime
import threading

# Configure the Reddit authentication information
client_id = "AxDDM4kYzBP1kgCWWxneGA"
client_secret = "cU5sRKcABBg0k9DTJ9lNuapcxz9K6w"
user_agent = "Sentiments Analysis/1.0 by JudeSimon01"

# Configure Kafka information
kafka_bootstrap_servers = "localhost:9092"
kafka_topic = "Reeditt"

# Create an instance of the Reddit API
reddit = praw.Reddit(
    client_id=client_id,
    client_secret=client_secret,
    user_agent=user_agent
)

# Create a Kafka producer instance
producer = KafkaProducer(
    bootstrap_servers=kafka_bootstrap_servers,
    value_serializer=lambda v: json.dumps(v).encode('utf-8')
)

# List of subreddits to monitor
subreddits = ["Ukraine","France","Bitcoin"]

# Set to store processed post IDs
processed_post_ids = set()

# Function to extract and publish messages to Kafka
def extract_and_publish_messages(subreddit_name):
    subreddit = reddit.subreddit(subreddit_name)
    for post in subreddit.new(limit=None):
        # Check if the post already exists
        if post.id not in processed_post_ids:
            # Add the post ID to the set of processed posts
            processed_post_ids.add(post.id)

            # Extract post information
            post_info = {
                'subreddit': post.subreddit.display_name,
                'author': post.author.name if post.author else None,
                'title': post.title,
                'content': post.selftext,
                'day': datetime.utcfromtimestamp(post.created_utc).strftime('%A'),
                'hour': datetime.utcfromtimestamp(post.created_utc).strftime('%H')
            }

            # Publish post information to Kafka
            producer.send(kafka_topic, value=post_info)

    print(f"Messages published to Kafka for subreddit: {subreddit_name}")

# Function to extract and publish messages for all subreddits
def extract_and_publish_all():
    threads = []
    for subreddit_name in subreddits:
        thread = threading.Thread(target=extract_and_publish_messages, args=(subreddit_name,))
        threads.append(thread)
        thread.start()

    # Wait for all threads to finish
    for thread in threads:
        thread.join()

# Schedule the function to run every 2 minutes
schedule.every(2).minutes.do(extract_and_publish_all)

# Main loop to execute the scheduler
while True:
    schedule.run_pending()
    time.sleep(1)

Messages published to Kafka for subreddit: France
Messages published to Kafka for subreddit: Bitcoin
Messages published to Kafka for subreddit: Ukraine
Messages published to Kafka for subreddit: France
Messages published to Kafka for subreddit: Ukraine
Messages published to Kafka for subreddit: Bitcoin
Messages published to Kafka for subreddit: France
Messages published to Kafka for subreddit: Bitcoin
Messages published to Kafka for subreddit: Ukraine
Messages published to Kafka for subreddit: France
Messages published to Kafka for subreddit: Bitcoin
Messages published to Kafka for subreddit: Ukraine
Messages published to Kafka for subreddit: France
Messages published to Kafka for subreddit: Bitcoin
Messages published to Kafka for subreddit: Ukraine
Messages published to Kafka for subreddit: France
Messages published to Kafka for subreddit: Ukraine
Messages published to Kafka for subreddit: Bitcoin
Messages published to Kafka for subreddit: Bitcoin
Messages published to Kafka for subre